## Working notebook for soil moisture map exploration
### This notebook analyzes the soil moisture map data over time for the entire area of interest (107 PHU)

This tool performs a time series trend analysis of the soil moisture maps. A linear model is applied to the time series of data and results in 2 outputs. The first output is the slope of the linear model. The slope indicates if the trend in soil moisture is negative or positive. These trends might be related to peatland management practices. The second output of the linear regression model is the p-value which indicates the significance of the model. The p-values range between 0 and 1. The closer the p-value output is to 0, the higher the model significance. If the p-value is high (closer to 0), the higher the probability that there is no linear relationship in the time series of soil moisture data. 

After the processing completes, download the outputs and check them in a GIS environment such as QGIS or ArcGIS. 

In [1]:
# INSTALL LIBRARY TO RUN R AND PYTHON AND BASH IN THE SAME NOTEBOOK
!pip install rpy2 --user -q

  Cache entry deserialization failed, entry ignored
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-OrcS4D/rpy2/


In [2]:
## ENABLE USE OF R AND PYTHON IN THE SAME NOTEBOOK
%load_ext rpy2.ipython


In [3]:
## CREATE FOLDER TO STORE SMM DATABASE
!mkdir -p ~/ws_idn_20190819/data/smm_phu/all_smm
## DOWNLOAD THE TIME SERIES OF SOIL MOISTURE MAPS


!wget -O ~/ws_idn_20190819/data/smm_phu/all_smm/all_phu_smm.tif https://www.dropbox.com/s/pe1fr05to5vuy0c/all_phu_smm.tif
!wget -O ~/ws_idn_20190819/data/smm_phu/all_smm/all_phu_smm.csv https://www.dropbox.com/s/hcbnvrjo6ys1ymi/all_phu_smm.csv
        

--2019-11-28 03:11:47--  https://www.dropbox.com/s/pe1fr05to5vuy0c/all_phu_smm.tif
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.1, 2620:100:6020:1::a27d:4001
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pe1fr05to5vuy0c/all_phu_smm.tif [following]
--2019-11-28 03:11:47--  https://www.dropbox.com/s/raw/pe1fr05to5vuy0c/all_phu_smm.tif
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf5e98ea0ce94e69d83abf2c236.dl.dropboxusercontent.com/cd/0/inline/AtMeMfYRfE2RBFnIX8W4zkdMmciw_iaeDmA3LuWjfzZPCK_CCxIcQYTxWt-gD5r4lTC3XYX4F8627MKBCznsWzuGrnprOBYIKiM4_Zz430pLKbPgu9RyMvFGtSz9BUbDy4w/file# [following]
--2019-11-28 03:11:47--  https://ucf5e98ea0ce94e69d83abf2c236.dl.dropboxusercontent.com/cd/0/inline/AtMeMfYRfE2RBFnIX8W4zkdMmciw_iaeDmA3LuWjfzZPCK_CCxIcQYTxWt-gD5r4lTC3XYX4F8627MKBCznsWzuGrnprOBYIKiM4_Z

In [ ]:
%%R
## LOAD THE PARAMETERS
source('~/ws_idn_20190819/scripts/s0_parameters.R')
library(ggplot2)
library(dplyr)
library(hrbrthemes)
library(leaflet)

In [ ]:
%%R
## READ THE SOIL MOISTURE MAP FOR ALL THE PHUS
smm_dir <- '~/ws_idn_20190819/data/smm_phu/all_smm/'
all.smm.ras <-  brick(paste0(smm_dir,'all_phu_smm.tif'))
all.smm.dat <-  read.csv(paste0(smm_dir,'all_phu_smm.csv'))
ts.date <- as.Date(unlist(all.smm.dat[,2]))

# plot(all.smm.ras[[99]])
# head(all.smm.dat[,1:3])
head(ts.date)

In [ ]:
# !Rscript /home/finegold/ws_idn_20190819_archive/scripts/smm_postprocessing/linear_model_smm_20191128.R


In [ ]:
%%R

NAvalue(all.smm.ras) <- 0

## calculate the slope
fun_slope <- function(y) { 
if(all(is.na(y))) {
  NA
} else {
  m = lm(y ~ ts.date, na.action=na.omit); summary(m)$coefficients[2] 
}
}

## calculate the p-value
fun_pvalue <- function(y) { 
if(all(is.na(y))) {
  NA
} else {
  m = lm(y ~ ts.date, na.action=na.omit); summary(m)$coefficients[8] 
}
}
## calculate the mean
fun_mean <- function(x) calc(x, fun = mean, na.rm = T)
## calculate the standard deviation
fun_stdv <- function(x) calc(x, fun = sd, na.rm = T)

# beginCluster()
# slope <- clusterR(all.smm.ras, calc, args=list(fun=fun_slope))
# pvalue <- clusterR(all.smm.ras, calc, args=list(fun=fun_pvalue))
# mean <- clusterR(all.smm.ras, fun_mean)
# stdv <- clusterR(all.smm.ras, fun_stdv)
# endCluster()

slope <-  calc(all.smm.ras, fun_slope)
pvalue <- calc(all.smm.ras,fun_pvalue)
mean <-   calc(all.smm.ras, fun = mean, na.rm = T)
stdv <-   calc(all.smm.ras, fun = sd, na.rm = T)

writeRaster(slope,paste0(smm_dir,"all_smm_slope.tif"),overwrite=T)
print('Completed linear regression slope map')
writeRaster(pvalue,paste0(smm_dir,"all_smm_pvalue.tif"),overwrite=T)
print('Completed linear regression signifance map')
writeRaster(mean,paste0(smm_dir,"all_smm_mean.tif"),overwrite=T)
print('Completed mean map')
writeRaster(stdv,paste0(smm_dir,"all_smm_stdv.tif"),overwrite=T)
print('Completed standard deviation map')
